# Protótipo
# Simulador Preditivo + Feature Engineering Automatizado

## Dados

In [1]:
import pandas as pd
import numpy as np

# Definindo os valores dos investimentos e os coeficientes para a regressão linear
num_semanas = 125
investimento_tv = np.random.randint(1000, 10000, size=num_semanas)  # Valores aleatórios para investimento em TV
investimento_radio = np.random.randint(500, 5000, size=num_semanas)  # Valores aleatórios para investimento em rádio
investimento_facebook = np.random.randint(200, 3000, size=num_semanas)  # Valores aleatórios para investimento no Facebook
investimento_google = np.random.randint(100, 2000, size=num_semanas)  # Valores aleatórios para investimento no Google

# Coeficientes para a regressão linear
coef_tv = 0.5
coef_radio = 0.3
coef_facebook = 0.2
coef_google = 0.1

# Calculando a quantidade de leads com base nos investimentos e nos coeficientes
leads = (investimento_tv * coef_tv) + (investimento_radio * coef_radio) + (investimento_facebook * coef_facebook) + (investimento_google * coef_google)

# Ajustando a quantidade de leads para ficar em torno de 500.000
fator_ajuste = 500000 / leads.sum()
leads *= fator_ajuste

# Criando a coluna de semana
semanas = pd.date_range(start='2022-01-01', periods=num_semanas, freq='W')

# Criando o DataFrame
df = pd.DataFrame({
    'semana': semanas,
    'investimento_tv': investimento_tv,
    'investimento_radio': investimento_radio,
    'investimento_facebook': investimento_facebook,
    'investimento_google': investimento_google,
    'leads': leads.astype(int)  # Convertendo para inteiros
})

# Exibindo o DataFrame
df


,semana,investimento_tv,investimento_radio,investimento_facebook,investimento_google,leads
0,2022-01-02,9971,3128,409,1154,6011
1,2022-01-09,6512,1866,1807,1814,4280
2,2022-01-16,8671,2931,2636,1956,5831
3,2022-01-23,2461,3481,1786,1998,2781
4,2022-01-30,2907,3806,1134,1796,2948
...,...,...,...,...,...,...
120,2024-04-21,6340,3185,2619,227,4588
121,2024-04-28,5804,4851,1255,359,4561
122,2024-05-05,8917,3843,2222,1316,6076
123,2024-05-12,6492,3405,1407,372,4504


## Divisão Treino - Teste

In [2]:
# Divisão de Dados:

from sklearn.model_selection import train_test_split

# Dividir os dados em conjuntos de treinamento e teste (exemplo: 80% treinamento, 20% teste)
X = df.drop(['leads', 'semana'], axis=1)  # Variáveis independentes
y = df['leads']  # Variável alvo

## Pipeline Automatizado - Feature Engineering

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# X é o conjunto de dados de características e y seja é o conjunto de dados de destino

# Divida seus dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crie um pipeline com as etapas desejadas
pipeline = Pipeline([
    ('scaling', StandardScaler(with_mean=False)),                  # Normalização dos dados
    ('polynomial_features', PolynomialFeatures()),  # Criação de features polinomiais
    ('feature_selection', SelectKBest(score_func=f_regression, k=5)),  # Seleção das melhores features
    ('model', LinearRegression())                   # Modelo de regressão linear
])

# Treine o pipeline no conjunto de treinamento
pipeline.fit(X_train, y_train)

# Avalie o desempenho do pipeline no conjunto de teste
score = pipeline.score(X_test, y_test)
print("Score do modelo:", score)


Score do modelo: 0.9797090493682469


In [4]:
pipeline

Pipeline(steps=[('scaling', StandardScaler(with_mean=False)),
                ('polynomial_features', PolynomialFeatures()),
                ('feature_selection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x3054ac220>)),
                ('model', LinearRegression())])

## Simulador com Sliders

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interactive, widgets
from IPython.display import display

# Defina uma função para calcular os leads mensais
def predict_leads(investimento_tv, investimento_radio, investimento_facebook, investimento_google):
    # Use o modelo treinado para fazer predições
    X_new = [[investimento_tv, investimento_radio, investimento_facebook, investimento_google]]
    leads_pred = pipeline.predict(X_new)
    return leads_pred[0]

# Defina uma função para plotar os resultados da predição
def plot_prediction(investimento_tv, investimento_radio, investimento_facebook, investimento_google):
    # Calcular o orçamento total
    budget_total = investimento_tv + investimento_radio + investimento_facebook + investimento_google
    
    # Calcular os leads mensais
    leads_pred = predict_leads(investimento_tv, investimento_radio, investimento_facebook, investimento_google)
    
    # Exibir os resultados
    print("Leads mensais =", int(leads_pred))
    print("Budget total = R$ {:,.2f}".format(budget_total))

# Calcular os limites dos sliders
tv_mean = df['investimento_tv'].mean()
tv_std = df['investimento_tv'].std()
tv_min = max(0, int(tv_mean - 2 * tv_std))  # Garantir que o valor mínimo não seja negativo
tv_max = int(tv_mean + 2 * tv_std)

radio_mean = df['investimento_radio'].mean()
radio_std = df['investimento_radio'].std()
radio_min = max(0, int(radio_mean - 2 * radio_std))
radio_max = int(radio_mean + 2 * radio_std)

facebook_mean = df['investimento_facebook'].mean()
facebook_std = df['investimento_facebook'].std()
facebook_min = max(0, int(facebook_mean - 2 * facebook_std))
facebook_max = int(facebook_mean + 2 * facebook_std)

google_mean = df['investimento_google'].mean()
google_std = df['investimento_google'].std()
google_min = max(0, int(google_mean - 2 * google_std))
google_max = int(google_mean + 2 * google_std)

# Criar sliders com os limites calculados
slider_tv = widgets.IntSlider(value=tv_mean, min=tv_min, max=tv_max, step=100, description='TV:')
slider_radio = widgets.IntSlider(value=radio_mean, min=radio_min, max=radio_max, step=100, description='Rádio:')
slider_facebook = widgets.IntSlider(value=facebook_mean, min=facebook_min, max=facebook_max, step=100, description='Facebook:')
slider_google = widgets.IntSlider(value=google_mean, min=google_min, max=google_max, step=100, description='Google:')

# Aplicar estilo aos sliders
style = {'description_width': '70px', 'handle_color': '#428bca'}
slider_tv.style = style
slider_radio.style = style
slider_facebook.style = style
slider_google.style = style

# Registrar a função de atualização para ser chamada sempre que os sliders forem alterados
interactive_plot = interactive(plot_prediction, investimento_tv=slider_tv, investimento_radio=slider_radio,
                               investimento_facebook=slider_facebook, investimento_google=slider_google)

# Calcular os valores estatísticos
stats_df = pd.DataFrame({
    'Coluna': ['TV', 'Rádio', 'Facebook', 'Google'],
    'Mínimo': [tv_min, radio_min, facebook_min, google_min],
    'Média': [tv_mean, radio_mean, facebook_mean, google_mean],
    'Mediana': [df['investimento_tv'].median(), df['investimento_radio'].median(), 
                df['investimento_facebook'].median(), df['investimento_google'].median()],
    'Máximo': [tv_max, radio_max, facebook_max, google_max]
})

# Exibir a tabela
display(stats_df)

# Exibir a interface interativa
display(interactive_plot)


,Coluna,Mínimo,Média,Mediana,Máximo
0,TV,620,5672.088,5954.0,10724
1,Rádio,25,2705.896,2700.0,5386
2,Facebook,0,1575.744,1570.0,3184
3,Google,31,1097.568,1122.0,2164


interactive(children=(IntSlider(value=5672, description='TV:', max=10724, min=620, step=100, style=SliderStyle…

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interactive, widgets
from IPython.display import display

# Defina uma função para calcular os leads mensais
def predict_leads(investimento_tv, investimento_radio, investimento_facebook, investimento_google):
    # Use o modelo treinado para fazer predições
    X_new = [[investimento_tv, investimento_radio, investimento_facebook, investimento_google]]
    leads_pred = pipeline.predict(X_new)
    return leads_pred[0]

# Defina uma função para plotar os resultados da predição
def plot_prediction(investimento_tv, investimento_radio, investimento_facebook, investimento_google):
    # Calcular o orçamento total
    budget_total = investimento_tv + investimento_radio + investimento_facebook + investimento_google
    
    # Calcular os leads mensais
    leads_pred = predict_leads(investimento_tv, investimento_radio, investimento_facebook, investimento_google)
    
     # Configurações de estilo
    plt.rcParams.update({'font.size': 14})  # Define o tamanho da fonte
    
    # Criar a figura e os subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))  # Cria uma grade de subplots
    
    # Plotar o budget total no primeiro subplot
    ax1.bar(['Budget Total'], [budget_total], color='skyblue')
    ax1.text('Budget Total', budget_total + 0.05 * budget_total, f'R$ {budget_total:,.2f}', ha='center', va='bottom')
    ax1.set_title('Budget Total')
    ax1.set_ylabel('Valor')
    ax1.set_ylim(0, 25000)
    
    # Plotar os leads mensais no segundo subplot
    ax2.bar(['Leads Mensais'], [leads_pred], color='orange')
    ax2.text('Leads Mensais', leads_pred + 0.05 * leads_pred, f'{int(leads_pred):,}', ha='center', va='bottom')
    ax2.set_title('Leads Mensais')
    ax2.set_ylabel('Valor')
    ax2.set_ylim(0, 10000)
    
    # Exibir os subplots
    plt.tight_layout()  # Ajusta automaticamente a posição dos subplots para evitar sobreposição
    plt.show()


# Calcular os limites dos sliders
tv_mean = df['investimento_tv'].mean()
tv_std = df['investimento_tv'].std()
tv_min = max(0, int(tv_mean - 2 * tv_std))  # Garantir que o valor mínimo não seja negativo
tv_max = int(tv_mean + 2 * tv_std)

radio_mean = df['investimento_radio'].mean()
radio_std = df['investimento_radio'].std()
radio_min = max(0, int(radio_mean - 2 * radio_std))
radio_max = int(radio_mean + 2 * radio_std)

facebook_mean = df['investimento_facebook'].mean()
facebook_std = df['investimento_facebook'].std()
facebook_min = max(0, int(facebook_mean - 2 * facebook_std))
facebook_max = int(facebook_mean + 2 * facebook_std)

google_mean = df['investimento_google'].mean()
google_std = df['investimento_google'].std()
google_min = max(0, int(google_mean - 2 * google_std))
google_max = int(google_mean + 2 * google_std)

# Criar sliders com os limites calculados
slider_tv = widgets.IntSlider(value=tv_mean, min=tv_min, max=tv_max, step=100, description='TV:')
slider_radio = widgets.IntSlider(value=radio_mean, min=radio_min, max=radio_max, step=100, description='Rádio:')
slider_facebook = widgets.IntSlider(value=facebook_mean, min=facebook_min, max=facebook_max, step=100, description='Facebook:')
slider_google = widgets.IntSlider(value=google_mean, min=google_min, max=google_max, step=100, description='Google:')

# Aplicar estilo aos sliders
style = {'description_width': '70px', 'handle_color': '#428bca'}
slider_tv.style = style
slider_radio.style = style
slider_facebook.style = style
slider_google.style = style

# Registrar a função de atualização para ser chamada sempre que os sliders forem alterados
interactive_plot = interactive(plot_prediction, investimento_tv=slider_tv, investimento_radio=slider_radio,
                               investimento_facebook=slider_facebook, investimento_google=slider_google)

# Calcular os valores estatísticos
stats_df = pd.DataFrame({
    'Coluna': ['TV', 'Rádio', 'Facebook', 'Google'],
    'Mínimo': [tv_min, radio_min, facebook_min, google_min],
    'Média': [tv_mean, radio_mean, facebook_mean, google_mean],
    'Mediana': [df['investimento_tv'].median(), df['investimento_radio'].median(), 
                df['investimento_facebook'].median(), df['investimento_google'].median()],
    'Máximo': [tv_max, radio_max, facebook_max, google_max]
})

# Exibir a tabela
display(stats_df)

# Exibir a interface interativa
display(interactive_plot)


,Coluna,Mínimo,Média,Mediana,Máximo
0,TV,620,5672.088,5954.0,10724
1,Rádio,25,2705.896,2700.0,5386
2,Facebook,0,1575.744,1570.0,3184
3,Google,31,1097.568,1122.0,2164


interactive(children=(IntSlider(value=5672, description='TV:', max=10724, min=620, step=100, style=SliderStyle…

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interactive, widgets
from IPython.display import display

# Defina uma função para calcular os leads mensais
def predict_leads(investimento_tv, investimento_radio, investimento_facebook, investimento_google):
    # Use o modelo treinado para fazer predições
    X_new = [[investimento_tv, investimento_radio, investimento_facebook, investimento_google]]
    leads_pred = pipeline.predict(X_new)
    return leads_pred[0]

# Defina uma função para plotar os resultados da predição
def plot_prediction(investimento_tv, investimento_radio, investimento_facebook, investimento_google):
    # Calcular os orçamentos para cada tipo de investimento
    budgets = [investimento_tv, investimento_radio, investimento_facebook, investimento_google]
    categories = ['TV', 'Rádio', 'Facebook', 'Google']
    
    # Calcular os leads mensais
    leads_pred = predict_leads(investimento_tv, investimento_radio, investimento_facebook, investimento_google)
    
    # Configurações de estilo
    plt.rcParams.update({'font.size': 14})  # Define o tamanho da fonte
    
    # Criar a figura e os subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))  # Cria uma grade de subplots
    
    # Plotar os orçamentos para cada tipo de investimento no primeiro subplot
    ax1.bar(categories, budgets, color=['blue', 'green', 'red', 'orange'])
    ax1.set_title('Orçamento por Tipo de Investimento')
    ax1.set_ylabel('Valor')
    ax1.set_ylim(0, 15000)
    
    # Plotar os leads mensais no segundo subplot
    ax2.bar(['Leads Mensais'], [leads_pred], color='orange')
    ax2.text('Leads Mensais', leads_pred + 0.05 * leads_pred, f'{int(leads_pred):,}', ha='center', va='bottom')
    ax2.set_title('Leads Mensais')
    ax2.set_ylabel('Valor')
    ax2.set_ylim(0, 15000)
    
    # Exibir os subplots
    plt.tight_layout()  # Ajusta automaticamente a posição dos subplots para evitar sobreposição
    plt.show()


# Calcular os limites dos sliders
tv_mean = df['investimento_tv'].mean()
tv_std = df['investimento_tv'].std()
tv_min = max(0, int(tv_mean - 2 * tv_std))  # Garantir que o valor mínimo não seja negativo
tv_max = int(tv_mean + 2 * tv_std)

radio_mean = df['investimento_radio'].mean()
radio_std = df['investimento_radio'].std()
radio_min = max(0, int(radio_mean - 2 * radio_std))
radio_max = int(radio_mean + 2 * radio_std)

facebook_mean = df['investimento_facebook'].mean()
facebook_std = df['investimento_facebook'].std()
facebook_min = max(0, int(facebook_mean - 2 * facebook_std))
facebook_max = int(facebook_mean + 2 * facebook_std)

google_mean = df['investimento_google'].mean()
google_std = df['investimento_google'].std()
google_min = max(0, int(google_mean - 2 * google_std))
google_max = int(google_mean + 2 * google_std)

# Criar sliders com os limites calculados
slider_tv = widgets.IntSlider(value=tv_mean, min=tv_min, max=tv_max, step=100, description='TV:')
slider_radio = widgets.IntSlider(value=radio_mean, min=radio_min, max=radio_max, step=100, description='Rádio:')
slider_facebook = widgets.IntSlider(value=facebook_mean, min=facebook_min, max=facebook_max, step=100, description='Facebook:')
slider_google = widgets.IntSlider(value=google_mean, min=google_min, max=google_max, step=100, description='Google:')

# Aplicar estilo aos sliders
style = {'description_width': '70px', 'handle_color': '#428bca'}
slider_tv.style = style
slider_radio.style = style
slider_facebook.style = style
slider_google.style = style

# Registrar a função de atualização para ser chamada sempre que os sliders forem alterados
interactive_plot = interactive(plot_prediction, investimento_tv=slider_tv, investimento_radio=slider_radio,
                               investimento_facebook=slider_facebook, investimento_google=slider_google)

# Calcular os valores estatísticos
stats_df = pd.DataFrame({
    'Coluna': ['TV', 'Rádio', 'Facebook', 'Google'],
    'Mínimo': [tv_min, radio_min, facebook_min, google_min],
    'Média': [tv_mean, radio_mean, facebook_mean, google_mean],
    'Mediana': [df['investimento_tv'].median(), df['investimento_radio'].median(), 
                df['investimento_facebook'].median(), df['investimento_google'].median()],
    'Máximo': [tv_max, radio_max, facebook_max, google_max]
})

# Exibir a tabela
display(stats_df)

# Exibir a interface interativa
display(interactive_plot)


,Coluna,Mínimo,Média,Mediana,Máximo
0,TV,620,5672.088,5954.0,10724
1,Rádio,25,2705.896,2700.0,5386
2,Facebook,0,1575.744,1570.0,3184
3,Google,31,1097.568,1122.0,2164


interactive(children=(IntSlider(value=5672, description='TV:', max=10724, min=620, step=100, style=SliderStyle…

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interactive, widgets
from IPython.display import display

# Defina uma função para calcular os leads mensais
def predict_leads(investimento_tv, investimento_radio, investimento_facebook, investimento_google):
    # Use o modelo treinado para fazer predições
    X_new = [[investimento_tv, investimento_radio, investimento_facebook, investimento_google]]
    leads_pred = pipeline.predict(X_new)
    return leads_pred[0]

# Defina uma função para plotar os resultados da predição
def plot_prediction(investimento_tv, investimento_radio, investimento_facebook, investimento_google):
    # Calcular os orçamentos para cada tipo de investimento
    budgets = [investimento_tv, investimento_radio, investimento_facebook, investimento_google]
    categories = ['TV', 'Rádio', 'Facebook', 'Google']
    
    # Calcular o budget total
    budget_total = sum(budgets)
    
    # Calcular os leads mensais
    leads_pred = predict_leads(investimento_tv, investimento_radio, investimento_facebook, investimento_google)
    
    # Configurações de estilo
    plt.rcParams.update({'font.size': 14})  # Define o tamanho da fonte
    
    # Criar a figura e os subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))  # Cria uma grade de subplots
    
    # Plotar os orçamentos para cada tipo de investimento e o budget total no primeiro subplot
    bars = ax1.bar(categories + ['Total'], budgets + [budget_total], color=['blue', 'green', 'red', 'orange', 'gray'])
    ax1.set_title('Orçamento por Tipo de Investimento')
    ax1.set_ylabel('Valor')
    ax1.set_ylim(0, 40000)
    
    # Adicionar texto acima de cada barra relacionada ao budget com os valores em reais
    for bar, budget in zip(bars, budgets + [budget_total]):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width() / 2, height + 0.05 * height, f'R$ {budget:,.2f}', ha='center', va='bottom', fontsize=10)
    
    # Plotar os leads mensais no segundo subplot
    ax2.bar(['Leads Mensais'], [leads_pred], color='orange')
    ax2.text('Leads Mensais', leads_pred + 0.05 * leads_pred, f'{int(leads_pred):,}', ha='center', va='bottom')
    ax2.set_title('Leads Mensais')
    ax2.set_ylabel('Valor')
    ax2.set_ylim(0, 20000)
    
    # Exibir os subplots
    plt.tight_layout()  # Ajusta automaticamente a posição dos subplots para evitar sobreposição
    plt.show()


# Calcular os limites dos sliders
tv_mean = df['investimento_tv'].mean()
tv_std = df['investimento_tv'].std()
tv_min = max(0, int(tv_mean - 5 * tv_std))  # Garantir que o valor mínimo não seja negativo
tv_max = int(tv_mean + 5 * tv_std)

radio_mean = df['investimento_radio'].mean()
radio_std = df['investimento_radio'].std()
radio_min = max(0, int(radio_mean - 5 * radio_std))
radio_max = int(radio_mean + 5 * radio_std)

facebook_mean = df['investimento_facebook'].mean()
facebook_std = df['investimento_facebook'].std()
facebook_min = max(0, int(facebook_mean - 5 * facebook_std))
facebook_max = int(facebook_mean + 5 * facebook_std)

google_mean = df['investimento_google'].mean()
google_std = df['investimento_google'].std()
google_min = max(0, int(google_mean - 5 * google_std))
google_max = int(google_mean + 5 * google_std)

# Criar sliders com os limites calculados
slider_tv = widgets.IntSlider(value=tv_mean, min=tv_min, max=tv_max, step=100, description='TV:')
slider_radio = widgets.IntSlider(value=radio_mean, min=radio_min, max=radio_max, step=100, description='Rádio:')
slider_facebook = widgets.IntSlider(value=facebook_mean, min=facebook_min, max=facebook_max, step=100, description='Facebook:')
slider_google = widgets.IntSlider(value=google_mean, min=google_min, max=google_max, step=100, description='Google:')

# Aplicar estilo aos sliders
style = {'description_width': '70px', 'handle_color': '#428bca'}
slider_tv.style = style
slider_radio.style = style
slider_facebook.style = style
slider_google.style = style

# Registrar a função de atualização para ser chamada sempre que os sliders forem alterados
interactive_plot = interactive(plot_prediction, investimento_tv=slider_tv, investimento_radio=slider_radio,
                               investimento_facebook=slider_facebook, investimento_google=slider_google)

# Calcular os valores estatísticos
stats_df = pd.DataFrame({
    'Coluna': ['TV', 'Rádio', 'Facebook', 'Google'],
    'Mínimo': [tv_min, radio_min, facebook_min, google_min],
    'Média': [tv_mean, radio_mean, facebook_mean, google_mean],
    'Mediana': [df['investimento_tv'].median(), df['investimento_radio'].median(), 
                df['investimento_facebook'].median(), df['investimento_google'].median()],
    'Máximo': [tv_max, radio_max, facebook_max, google_max]
})

# Exibir a tabela
display(stats_df)

# Exibir a interface interativa
display(interactive_plot)


,Coluna,Mínimo,Média,Mediana,Máximo
0,TV,0,5672.088,5954.0,18301
1,Rádio,0,2705.896,2700.0,9406
2,Facebook,0,1575.744,1570.0,5598
3,Google,0,1097.568,1122.0,3763


interactive(children=(IntSlider(value=5672, description='TV:', max=18301, step=100, style=SliderStyle(descript…